***GENERATED CODE FOR college PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'University', 'transformation_label': 'String Indexer'}], 'feature': 'University', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '158', 'mean': '', 'stddev': '', 'min': 'Albertus Magnus College', 'max': 'York College of Pennsylvania', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'University', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('University')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Private', 'transformation_label': 'String Indexer'}], 'feature': 'Private', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '158', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Private', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Private')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        kmeans_model = kmeans(df=spark_DF, TrainingPercent=trainingPercent)


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run collegeHooks.ipynb
try:
	sourcePreExecutionHook()

	collegedbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/Clustering/College.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")
	sourcePostExecutionHook(collegedbfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
%run collegeHooks.ipynb
try:
	transformationPreExecutionHook()

	collegeautofe = TransformationMain.run(collegedbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "University", "transformation_label": "String Indexer"}], "feature": "University", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "158", "mean": "", "stddev": "", "min": "Albertus Magnus College", "max": "York College of Pennsylvania", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Private", "transformation_label": "String Indexer"}], "feature": "Private", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "158", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Apps", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "2756.54", "stddev": "3447.14", "min": "81", "max": "19873", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Accept", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "1879.82", "stddev": "2122.16", "min": "72", "max": "12414", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Enroll", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "755.72", "stddev": "892.01", "min": "46", "max": "5329", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Top10perc", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "27.12", "stddev": "17.52", "min": "3", "max": "95", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Top25perc", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "55.67", "stddev": "20.15", "min": "17", "max": "100", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_Undergrad", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "3669.34", "stddev": "5018.65", "min": "139", "max": "30017", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "P_Undergrad", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "841.74", "stddev": "1220.89", "min": "1", "max": "7443", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Outstate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "10448.76", "stddev": "4047.37", "min": "2700", "max": "21700", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Room_Board", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "4339.44", "stddev": "1081.72", "min": "2190", "max": "7000", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Books", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "525.42", "stddev": "126.49", "min": "200", "max": "1200", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Personal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "1321.54", "stddev": "640.89", "min": "250", "max": "3700", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PhD", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "71.86", "stddev": "16.66", "min": "10", "max": "99", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Terminal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "78.59", "stddev": "15.33", "min": "24", "max": "100", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SF_Ratio", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "158", "mean": "14.07", "stddev": "4.24", "min": "2.9", "max": "39.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "perc_alumni", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "22.81", "stddev": "11.97", "min": "1", "max": "60", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Expend", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "9719.03", "stddev": "5117.47", "min": "3186", "max": "42926", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Grad_Rate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "158", "mean": "65.26", "stddev": "17.97", "min": "10", "max": "118", "missing": "0"}}, {"feature": "University_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "158", "mean": "78.5", "stddev": "45.75", "min": "0.0", "max": "157.0", "missing": "0"}}, {"feature": "Private_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "158", "mean": "0.23", "stddev": "0.42", "min": "0", "max": "1", "missing": "0"}}]}))

	transformationPostExecutionHook(collegeautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
%run collegeHooks.ipynb
try:
	mlPreExecutionHook()

	collegeclustering = Clustering.run(collegeautofe,spark,json.dumps( {"autoClustering": 0, "orignalfile": "/Demo/Clustering/College.csv", "distinct_column": "Private_stringindexer", "model": "Kmeans", "TrainingPercent": 100, "run_id": "8b06a912ad654be6b9c2626664b4ef7b", "model_id": "5ef086dc9385da042b5b3db9"}))

	mlPostExecutionHook(collegeclustering)

except Exception as ex: 
	logging.error(ex)
